In [3]:
import pandas as pd

file_path = '../../data/school_health_preprocessed.csv'

try:
    data = pd.read_csv(file_path, encoding='utf-8')
    print("데이터 로드 성공!")
except FileNotFoundError:
    print("파일을 찾을 수 없습니다. 경로를 확인하세요.")


print(data[['하루30분이상운동']].isnull().sum())

cleaned_data = data.dropna(subset=['BMI', '하루30분이상운동'])
print(f"Cleaned data size: {cleaned_data.shape}")

# 성별로 데이터 분리
male_data = cleaned_data[cleaned_data['성별'] == '남']
female_data = cleaned_data[cleaned_data['성별'] == '여']


데이터 로드 성공!
하루30분이상운동    35081
dtype: int64
Cleaned data size: (49790, 111)


In [4]:
import pandas as pd

# 필요한 열 추출
df = data[['BMI', '주3회이상운동', '학교급', '성별', '하루30분이상운동']]

# '주3회이상운동' 관련 데이터 결측치 제거
cleaned_data_exercise_3times = df.dropna(subset=['BMI', '주3회이상운동', '학교급', '성별'])
print(f"Cleaned data size for 주3회이상운동: {cleaned_data_exercise_3times.shape}")
print(cleaned_data_exercise_3times.head())

# '주3회이상운동' 매핑 : 초등학생 대상
exercise_mapping_3times = {
    1.0: '예',
    2.0: '아니오'
}

cleaned_data_exercise_3times['주 3회 운동여부'] = cleaned_data_exercise_3times['주3회이상운동'].map(exercise_mapping_3times)
print("\n[주3회이상운동 매핑 결과 (초등학생 대상)]")
print(cleaned_data_exercise_3times[['BMI', '주3회이상운동', '주 3회 운동여부','성별','학교급']].head())

# '하루30분이상운동' 관련 데이터 결측치 제거
cleaned_data_exercise_30min = df.dropna(subset=['BMI', '하루30분이상운동', '학교급', '성별'])
print(f"Cleaned data size for 하루30분이상운동: {cleaned_data_exercise_30min.shape}")
print(cleaned_data_exercise_30min.head())

# '하루30분이상운동' 매핑 : 중, 고등학생 대상
exercise_mapping_30min = {
    1.0: '거의 안함',
    2.0: '주에 1~2회',
    3.0: '주에 3~4회',
    4.0: '주에 5회 이상'
}

cleaned_data_exercise_30min['운동빈도'] = cleaned_data_exercise_30min['하루30분이상운동'].map(exercise_mapping_30min)
print("\n[하루30분이상운동 매핑 결과 (중, 고등학생 대상)]")
print(cleaned_data_exercise_30min[['BMI', '하루30분이상운동', '운동빈도','성별','학교급']].head())


Cleaned data size for 주3회이상운동: (32715, 5)
         BMI  주3회이상운동 학교급 성별  하루30분이상운동
0  15.667728      2.0   초  여        NaN
1  15.644444      2.0   초  여        NaN
2  15.159148      2.0   초  여        NaN
3  22.701628      2.0   초  여        NaN
4  14.870556      1.0   초  여        NaN

[주3회이상운동 매핑 결과 (초등학생 대상)]
         BMI  주3회이상운동 주 3회 운동여부 성별 학교급
0  15.667728      2.0       아니오  여   초
1  15.644444      2.0       아니오  여   초
2  15.159148      2.0       아니오  여   초
3  22.701628      2.0       아니오  여   초
4  14.870556      1.0         예  여   초
Cleaned data size for 하루30분이상운동: (49790, 5)
             BMI  주3회이상운동 학교급 성별  하루30분이상운동
29623  15.238947      NaN   중  남        3.0
29624  20.342891      NaN   중  여        1.0
29625  24.689836      NaN   중  남        3.0
29626  20.877623      NaN   중  여        2.0
29627  17.201060      NaN   중  여        1.0

[하루30분이상운동 매핑 결과 (중, 고등학생 대상)]
             BMI  하루30분이상운동     운동빈도 성별 학교급
29623  15.238947        3.0  주에 3~4회  남   중
29624  20.342891        1.0  

/var/folders/qz/d378mfsj25g7bb288v8ls9yw0000gn/T/ipykernel_50661/1123144500.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data_exercise_3times['주 3회 운동여부'] = cleaned_data_exercise_3times['주3회이상운동'].map(exercise_mapping_3times)
/var/folders/qz/d378mfsj25g7bb288v8ls9yw0000gn/T/ipykernel_50661/1123144500.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data_exercise_30min['운동빈도'] = cleaned_data_exercise_30min['하루30분이상운동'].map(exercise_mapping_30min)


In [11]:
import pandas as pd
import statsmodels.api as sm

# 데이터 전처리
cleaned_data_exercise_30min['성별_encoded'] = cleaned_data_exercise_30min['성별'].map({'남': 1, '여': 0})
cleaned_data_exercise_30min['학교급_encoded'] = cleaned_data_exercise_30min['학교급'].map({'중': 1, '고': 2})
cleaned_data_exercise_30min['운동빈도_encoded'] = cleaned_data_exercise_30min['운동빈도'].map({
    '거의 안함': 1,
    '주에 1~2회': 2,
    '주에 3~4회': 3,
    '주에 5회 이상': 4
})

# 남중 그룹 필터링
male_middle_school = cleaned_data_exercise_30min[
    (cleaned_data_exercise_30min['성별_encoded'] == 1) &
    (cleaned_data_exercise_30min['학교급_encoded'] == 1)
]

# 독립 변수와 종속 변수
X = male_middle_school[['운동빈도_encoded']]
y = male_middle_school['BMI']

# 상수항 추가
X = sm.add_constant(X)

# 회귀 분석
model = sm.OLS(y, X)
results = model.fit()

# 결과 출력
print("\n[남중 그룹 회귀 분석 결과]")
print(results.summary())



[남중 그룹 회귀 분석 결과]
                            OLS Regression Results                            
Dep. Variable:                    BMI   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.3726
Date:                Tue, 10 Dec 2024   Prob (F-statistic):              0.542
Time:                        23:16:52   Log-Likelihood:                -32647.
No. Observations:               11685   AIC:                         6.530e+04
Df Residuals:                   11683   BIC:                         6.531e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           21.7300      0

/var/folders/qz/d378mfsj25g7bb288v8ls9yw0000gn/T/ipykernel_50661/2220593182.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data_exercise_30min['성별_encoded'] = cleaned_data_exercise_30min['성별'].map({'남': 1, '여': 0})
/var/folders/qz/d378mfsj25g7bb288v8ls9yw0000gn/T/ipykernel_50661/2220593182.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data_exercise_30min['학교급_encoded'] = cleaned_data_exercise_30min['학교급'].map({'중': 1, '고': 2})
/var/folders/qz/d378mfsj25g7bb288v8ls9yw0000gn/T/ipyke

In [12]:
# 남고 그룹 필터링
male_high_school = cleaned_data_exercise_30min[
    (cleaned_data_exercise_30min['성별_encoded'] == 1) &
    (cleaned_data_exercise_30min['학교급_encoded'] == 2)
]

# 독립 변수와 종속 변수
X = male_high_school[['운동빈도_encoded']]
y = male_high_school['BMI']

# 상수항 추가
X = sm.add_constant(X)

# 회귀 분석
model = sm.OLS(y, X)
results = model.fit()

# 결과 출력
print("\n[남고 그룹 회귀 분석 결과]")
print(results.summary())



[남고 그룹 회귀 분석 결과]
                            OLS Regression Results                            
Dep. Variable:                    BMI   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     19.96
Date:                Tue, 10 Dec 2024   Prob (F-statistic):           7.98e-06
Time:                        23:17:36   Log-Likelihood:                -36330.
No. Observations:               13348   AIC:                         7.266e+04
Df Residuals:                   13346   BIC:                         7.268e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           22.4523      0

In [13]:
# 여중 그룹 필터링
female_middle_school = cleaned_data_exercise_30min[
    (cleaned_data_exercise_30min['성별_encoded'] == 0) &
    (cleaned_data_exercise_30min['학교급_encoded'] == 1)
]

# 독립 변수와 종속 변수
X = female_middle_school[['운동빈도_encoded']]
y = female_middle_school['BMI']

# 상수항 추가
X = sm.add_constant(X)

# 회귀 분석
model = sm.OLS(y, X)
results = model.fit()

# 결과 출력
print("\n[여중 그룹 회귀 분석 결과]")
print(results.summary())



[여중 그룹 회귀 분석 결과]
                            OLS Regression Results                            
Dep. Variable:                    BMI   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     42.00
Date:                Tue, 10 Dec 2024   Prob (F-statistic):           9.49e-11
Time:                        23:17:43   Log-Likelihood:                -29861.
No. Observations:               11273   AIC:                         5.973e+04
Df Residuals:                   11271   BIC:                         5.974e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           20.3082      0

In [14]:
# 여고 그룹 필터링
female_high_school = cleaned_data_exercise_30min[
    (cleaned_data_exercise_30min['성별_encoded'] == 0) &
    (cleaned_data_exercise_30min['학교급_encoded'] == 2)
]

# 독립 변수와 종속 변수
X = female_high_school[['운동빈도_encoded']]
y = female_high_school['BMI']

# 상수항 추가
X = sm.add_constant(X)

# 회귀 분석
model = sm.OLS(y, X)
results = model.fit()

# 결과 출력
print("\n[여고 그룹 회귀 분석 결과]")
print(results.summary())



[여고 그룹 회귀 분석 결과]
                            OLS Regression Results                            
Dep. Variable:                    BMI   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     134.0
Date:                Tue, 10 Dec 2024   Prob (F-statistic):           7.56e-31
Time:                        23:17:51   Log-Likelihood:                -35486.
No. Observations:               13484   AIC:                         7.098e+04
Df Residuals:                   13482   BIC:                         7.099e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           20.9453      0

In [8]:
import pandas as pd
import statsmodels.api as sm

# 독립 변수 준비
cleaned_data_exercise_3times['운동여부_encoded'] = cleaned_data_exercise_3times['주 3회 운동여부'].map({'예': 1, '아니오': 0})  # 주 3회 운동 여부를 숫자로 변환

# 남초 그룹 필터링
male_group = cleaned_data_exercise_3times[cleaned_data_exercise_3times['성별'] == '남']

# 종속 변수 (BMI)와 독립 변수 설정 - 남초 그룹
X_male = male_group[['운동여부_encoded']]
y_male = male_group['BMI']

# 상수항 추가
X_male = sm.add_constant(X_male)

# 남초 그룹 회귀 모델 생성 및 적합
model_male = sm.OLS(y_male, X_male)
results_male = model_male.fit()

print("\n[남초 그룹 회귀 분석 결과]")
print(results_male.summary())





[남초 그룹 회귀 분석 결과]
                            OLS Regression Results                            
Dep. Variable:                    BMI   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     14.64
Date:                Tue, 10 Dec 2024   Prob (F-statistic):           0.000130
Time:                        23:12:27   Log-Likelihood:                -46526.
No. Observations:               16753   AIC:                         9.306e+04
Df Residuals:                   16751   BIC:                         9.307e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           19.3813      0

/var/folders/qz/d378mfsj25g7bb288v8ls9yw0000gn/T/ipykernel_50661/2931769307.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data_exercise_3times['운동여부_encoded'] = cleaned_data_exercise_3times['주 3회 운동여부'].map({'예': 1, '아니오': 0})  # 주 3회 운동 여부를 숫자로 변환


In [9]:
# 여초 그룹 필터링
female_group = cleaned_data_exercise_3times[cleaned_data_exercise_3times['성별'] == '여']

# 종속 변수 (BMI)와 독립 변수 설정 - 여초 그룹
X_female = female_group[['운동여부_encoded']]
y_female = female_group['BMI']

# 상수항 추가
X_female = sm.add_constant(X_female)

# 여초 그룹 회귀 모델 생성 및 적합
model_female = sm.OLS(y_female, X_female)
results_female = model_female.fit()

print("\n[여초 그룹 회귀 분석 결과]")
print(results_female.summary())


[여초 그룹 회귀 분석 결과]
                            OLS Regression Results                            
Dep. Variable:                    BMI   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                  0.001410
Date:                Tue, 10 Dec 2024   Prob (F-statistic):              0.970
Time:                        23:12:31   Log-Likelihood:                -42155.
No. Observations:               15962   AIC:                         8.431e+04
Df Residuals:                   15960   BIC:                         8.433e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           18.1660      0